In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [3]:
pwd

'/Users/aditya/Desktop/SER515/scrum_mates/myProject/Recommendation'

In [4]:
credits = pd.read_csv('test_data/credits.csv')
keywords = pd.read_csv('test_data/keywords.csv')
links = pd.read_csv('test_data/links.csv')
metaData = pd.read_csv('test_data/metaDataPreprocessed.csv')
ratings = pd.read_csv('test_data/ratings_small.csv')

In [5]:
metaData.head(10)

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,1,False,{},65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844.0,tt0113497,en,Jumanji,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602.0,tt0113228,en,Grumpier Old Men,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,3,False,{},16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357.0,tt0114885,en,Waiting to Exhale,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862.0,tt0113041,en,Father of the Bride Part II,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,5,False,{},60000000,"['Action', 'Crime', 'Drama', 'Thriller']",NaN,949.0,tt0113277,en,Heat,...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,6,False,{},58000000,"['Comedy', 'Romance']",NaN,11860.0,tt0114319,en,Sabrina,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,7,False,{},0,"['Action', 'Adventure', 'Drama', 'Family']",NaN,45325.0,tt0112302,en,Tom and Huck,...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,8,False,{},35000000,"['Action', 'Adventure', 'Thriller']",NaN,9091.0,tt0114576,en,Sudden Death,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"['Adventure', 'Action', 'Thriller']",http://www.mgm.com/view/movie/757/Goldeneye/,710.0,tt0113189,en,GoldenEye,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [6]:
metaData['year'] = pd.to_datetime(metaData['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [7]:
metaData.head(10)

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,1,False,{},65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844.0,tt0113497,en,Jumanji,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602.0,tt0113228,en,Grumpier Old Men,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995
3,3,False,{},16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357.0,tt0114885,en,Waiting to Exhale,...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862.0,tt0113041,en,Father of the Bride Part II,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995
5,5,False,{},60000000,"['Action', 'Crime', 'Drama', 'Thriller']",NaN,949.0,tt0113277,en,Heat,...,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995
6,6,False,{},58000000,"['Comedy', 'Romance']",NaN,11860.0,tt0114319,en,Sabrina,...,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995
7,7,False,{},0,"['Action', 'Adventure', 'Drama', 'Family']",NaN,45325.0,tt0112302,en,Tom and Huck,...,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995
8,8,False,{},35000000,"['Action', 'Adventure', 'Thriller']",NaN,9091.0,tt0114576,en,Sudden Death,...,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0,1995
9,9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"['Adventure', 'Action', 'Thriller']",http://www.mgm.com/view/movie/757/Goldeneye/,710.0,tt0113189,en,GoldenEye,...,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,1995


In [8]:
metaData.dtypes

Unnamed: 0                 int64
adult                       bool
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                       float64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                       bool
vote_average             float64
vote_count               float64
year                      object
dtype: object

In [9]:
metaData['year'].replace({'NaT': '0000'}, inplace=True)

In [10]:
metaData[['year']] = metaData[['year']].astype(int)

In [11]:
metaData[metaData.year > 2018].count()

Unnamed: 0               1
adult                    1
belongs_to_collection    1
budget                   1
genres                   1
homepage                 1
id                       1
imdb_id                  1
original_language        1
original_title           1
overview                 1
popularity               1
poster_path              1
production_companies     1
production_countries     1
release_date             1
revenue                  1
runtime                  1
spoken_languages         1
status                   1
tagline                  0
title                    1
video                    1
vote_average             1
vote_count               1
year                     1
dtype: int64

In [12]:
metaData[metaData.year > 2015].count()

Unnamed: 0               2140
adult                    2140
belongs_to_collection    2140
budget                   2140
genres                   2140
homepage                  836
id                       2140
imdb_id                  2140
original_language        2138
original_title           2140
overview                 2103
popularity               2140
poster_path              2132
production_companies     2140
production_countries     2140
release_date             2140
revenue                  2140
runtime                  2140
spoken_languages         2140
status                   2140
tagline                   907
title                    2140
video                    2140
vote_average             2140
vote_count               2140
year                     2140
dtype: int64

In [13]:
metaData['flag'] = metaData['year'].apply(lambda x: 1 if x > 2015 else 0)

In [14]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metaData['id'] = metaData['id'].astype('int')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [15]:
metaData.shape

(45443, 27)

In [16]:
metaData = metaData.merge(credits, on='id')
metaData = metaData.merge(keywords, on='id')

In [17]:
metaData.head(10)

,Unnamed: 0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,tagline,title,video,vote_average,vote_count,year,flag,cast,crew,keywords
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,NaN,Toy Story,False,7.7,5415.0,1995,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,1,False,{},65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,...,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,...,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,3,False,{},16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,...,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,...,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
5,5,False,{},60000000,"['Action', 'Crime', 'Drama', 'Thriller']",NaN,949,tt0113277,en,Heat,...,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,1995,0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
6,6,False,{},58000000,"['Comedy', 'Romance']",NaN,11860,tt0114319,en,Sabrina,...,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,1995,0,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...","[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
7,7,False,{},0,"['Action', 'Adventure', 'Drama', 'Family']",NaN,45325,tt0112302,en,Tom and Huck,...,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,1995,0,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[]
8,8,False,{},35000000,"['Action', 'Adventure', 'Thriller']",NaN,9091,tt0114576,en,Sudden Death,...,Terror goes into overtime.,Sudden Death,False,5.5,174.0,1995,0,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...","[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."
9,9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"['Adventure', 'Action', 'Thriller']",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,...,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0,1995,0,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...","[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."


In [18]:
metaData.columns

Index(['Unnamed: 0', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'year', 'flag', 'cast', 'crew',
       'keywords'],
      dtype='object')

In [19]:
metaData.shape

(46607, 30)

In [39]:
metaDataLinks = metaData[metaData.id.isin(links)]

In [40]:
metaDataLinks.shape

(46607, 30)

In [41]:
metaDataLinks['cast'] = metaDataLinks['cast'].apply(literal_eval)

In [42]:
metaDataLinks['crew'] = metaDataLinks['crew'].apply(literal_eval)

In [43]:
metaDataLinks['keywords'] = metaDataLinks['keywords'].apply(literal_eval)

In [44]:
metaDataLinks['cast_size'] = metaDataLinks['cast'].apply(lambda x: len(x))
metaDataLinks['crew_size'] = metaDataLinks['crew'].apply(lambda x: len(x))

In [45]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [46]:
metaDataLinks['director'] = metaDataLinks['crew'].apply(get_director)

In [47]:
metaDataLinks['cast'] = metaDataLinks['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [52]:
#metaDataLinks['cast'] = metaDataLinks['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
#metaDataLinks['cast']
#metaDataLinks['cast'] = metaDataLinks['cast'].str.lower()
metaDataLinks['director'] = metaDataLinks['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
metaDataLinks['director'] = metaDataLinks['director'].apply(lambda x: [x,x, x])

In [59]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [54]:
metaDataLinks['keywords'] = metaDataLinks['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [73]:
s = metaDataLinks.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
#s = s.value_counts()
#s = s[s > 1]
#stemmer = SnowballStemmer('english')

In [77]:
s.name = 'keyword'

s[:5]

0    ['jealousi', 'toy', 'boy', 'friendship', 'frie...
1    ['boardgam', 'disappear', "basedonchildren'sbo...
2    ['fish', 'bestfriend', 'duringcreditssting', '...
3    ['basedonnovel', 'interracialrelationship', 's...
4    ['babi', 'midlifecrisi', 'confid', 'age', 'dau...
Name: keyword, dtype: object

In [60]:
metaDataLinks['keywords'] = metaDataLinks['keywords'].apply(filter_keywords)
metaDataLinks['keywords'] = metaDataLinks['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
metaDataLinks['keywords'] = metaDataLinks['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [61]:
metaDataLinks['keywords'] = metaDataLinks['keywords'].astype(str)
metaDataLinks['cast'] = metaDataLinks['cast'].astype(str)
metaDataLinks['director'] = metaDataLinks['director'].astype(str)
metaDataLinks['genres'] = metaDataLinks['genres'].astype(str)

In [62]:
metaDataLinks['soup'] = metaDataLinks['keywords'] + metaDataLinks['cast'] + metaDataLinks['director'] + metaDataLinks['genres']

In [63]:
metaDataLinks['soup'] = metaDataLinks['soup'].apply(lambda x: ' '.join(x))

In [64]:
metaDataLinks['tagline'] = metaDataLinks['tagline'].fillna('')
metaDataLinks['description'] = metaDataLinks['overview'] + metaDataLinks['tagline']
metaDataLinks['description'] = metaDataLinks['description'].fillna('')['description'] = metaDataLinks['description'].fillna('')

In [65]:
metaDataLinks['final'] = metaDataLinks['soup'] + metaDataLinks['description']

In [66]:
metaDataLinks.shape

(46607, 36)

In [67]:
metaDataLinks = metaDataLinks.drop_duplicates(subset=['title'], keep='first')

In [68]:
metaDataLinks.shape

(42262, 36)

In [69]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(metaDataLinks['final'])

In [70]:
tfidf_matrix.shape

(42262, 1041106)

In [71]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [72]:
metaDataLinks = metaDataLinks.reset_index()
titles = metaDataLinks['title']
indices = pd.Series(metaDataLinks.index, index= metaDataLinks['title'])

In [73]:
metaDataLinks.columns

Index(['index', 'Unnamed: 0', 'adult', 'belongs_to_collection', 'budget',
       'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'year', 'flag', 'cast', 'crew',
       'keywords', 'cast_size', 'crew_size', 'director', 'soup', 'description',
       'final'],
      dtype='object')

In [105]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:]
    movie_indices = [i[0] for i in sim_scores]
    movies = metaDataLinks.iloc[movie_indices][['title','flag','id','imdb_id','year','popularity']]
    qualified = movies[movies.flag == 1]
    return qualified

In [109]:
get_recommendations('For Here or to Go?')

,title,flag,id,imdb_id,year,popularity
40297,11:55,1,394770,tt4250438,2017,0.722501
36881,Sunspring,1,401387,tt5794766,2016,1.124787
38232,It's All Good,1,422127,tt6103962,2016,0.349211
38298,Manchester by the Sea,1,334541,tt4034228,2016,12.821257
38615,Alienate,1,140491,tt2324928,2016,3.218530
36539,Girl in Woods,1,355598,tt2554842,2016,1.365827
33855,The Young Messiah,1,318850,tt1002563,2016,1.729199
41264,The Mars Generation,1,433878,tt6333096,2017,0.774638
41917,Decanted,1,425071,tt6085362,2016,0.060550
37022,The Call Up,1,388347,tt3923388,2016,4.060687
